In [100]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline

In [330]:
words = open("names.txt", "r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [331]:
len(words)

32033

In [332]:
chars = sorted(list(set("".join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [333]:
block_size = 4
X, Y = [], []

for w in words:
    # print(w)
    context = [0] * block_size
    for ch in w + ".":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context),'--->',itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X).to("mps")
Y = torch.tensor(Y).to("mps")

In [334]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 4]), torch.int64, torch.Size([228146]), torch.int64)

In [335]:
X

tensor([[ 0,  0,  0,  0],
        [ 0,  0,  0,  5],
        [ 0,  0,  5, 13],
        ...,
        [ 0, 26, 26, 25],
        [26, 26, 25, 26],
        [26, 25, 26, 24]], device='mps:0')

In [336]:
Y

tensor([ 5, 13, 13,  ..., 26, 24,  0], device='mps:0')

In [337]:
C = torch.randn((27, 2)).to("mps")

In [338]:
C[5]

tensor([-2.1333,  0.0041], device='mps:0')

In [339]:
# F.one_hot(torch.tensor(5) , num_classes=27).float() @ C (same as above)

In [340]:
C[X]

tensor([[[-6.8541e-01,  1.7445e+00],
         [-6.8541e-01,  1.7445e+00],
         [-6.8541e-01,  1.7445e+00],
         [-6.8541e-01,  1.7445e+00]],

        [[-6.8541e-01,  1.7445e+00],
         [-6.8541e-01,  1.7445e+00],
         [-6.8541e-01,  1.7445e+00],
         [-2.1333e+00,  4.0594e-03]],

        [[-6.8541e-01,  1.7445e+00],
         [-6.8541e-01,  1.7445e+00],
         [-2.1333e+00,  4.0594e-03],
         [ 1.7507e+00,  1.8410e+00]],

        ...,

        [[-6.8541e-01,  1.7445e+00],
         [ 1.0951e-03, -4.4056e-01],
         [ 1.0951e-03, -4.4056e-01],
         [ 1.1320e+00,  8.4298e-01]],

        [[ 1.0951e-03, -4.4056e-01],
         [ 1.0951e-03, -4.4056e-01],
         [ 1.1320e+00,  8.4298e-01],
         [ 1.0951e-03, -4.4056e-01]],

        [[ 1.0951e-03, -4.4056e-01],
         [ 1.1320e+00,  8.4298e-01],
         [ 1.0951e-03, -4.4056e-01],
         [-1.0914e-01, -5.9087e-01]]], device='mps:0')

In [341]:
C[X].shape

torch.Size([228146, 4, 2])

In [342]:
X[13, 2]

tensor(0, device='mps:0')

In [343]:
C[X][13, 2]

tensor([-0.6854,  1.7445], device='mps:0')

In [344]:
emb = C[X]
emb.shape

torch.Size([228146, 4, 2])

In [345]:
W1 = torch.randn((6, 100)).to("mps")
b1 = torch.randn(100).to("mps")

In [351]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

RuntimeError: shape '[-1, 6]' is invalid for input of size 1825168

In [352]:
# torch.cat([emb[: , 0, :] ,emb[: , 1 ,:] ,emb[: , 2 ,:]] , 1).shape

In [353]:
# torch.cat(torch.unbind(emb , 1) , 1).shape ## same as above

In [354]:
h.shape

torch.Size([1, 300])

In [355]:
W2 = torch.randn((100, 27)).to("mps")
b2 = torch.randn(27).to("mps")

In [356]:
logits = h @ W2 + b2

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x300 and 100x27)

In [357]:
logits.shape

torch.Size([1, 27])

In [358]:
counts = logits.exp()

In [359]:
prob = counts / counts.sum(1, keepdim=True)

In [360]:
prob.shape

torch.Size([1, 27])

In [361]:
prob[0].sum()

tensor(1., device='mps:0', grad_fn=<SumBackward0>)

In [362]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [228146]

In [363]:
torch.arange(32).to("mps")

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
       device='mps:0')

In [364]:
X.shape, Y.shape

(torch.Size([228146, 4]), torch.Size([228146]))

In [365]:
g = torch.Generator().manual_seed(2147483647)
C = torch.rand((27, 2), generator=g).to("mps")
W1 = torch.rand((6, 100), generator=g).to("mps")
b1 = torch.rand(100, generator=g).to("mps")
W2 = torch.randn((100, 27), generator=g).to("mps")
b2 = torch.randn(27, generator=g).to("mps")
parameters = [C, W1, b1, W2, b2]

In [366]:
sum(p.nelement() for p in parameters)  # number of parameters in total

3481

In [367]:
for p in parameters:
    p.requires_grad = True

In [368]:
# lre = torch.linspace(-3 , 0 , 1000)
# lrs = 10**lre
# lrs

In [371]:
# lri = []
# lossi = []
for i in range(1000):

    ix = torch.randint(0, X.shape[0], (32,))
    emb = C[X[ix]]  # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)  # (32, 100)
    logits = h @ W2 + b2  # (32, 27)
    # counts = logits. exp()
    # prob = counts / counts.sum(1, keepdims=True)n 
    # loss = -prob[torch.arange (32), Y].log().mean()
    hah = F.cross_entropy(logits, Y[ix])  # same as above
    print(hah.item())
    i += 1
    for p in parameters:
        p.grad = None

    hah.backward()
    # lr = lrs[i]
    for p in parameters:
        p.data += -0.1 * p.grad
        
    # lri.append(lre[i])
    # lossi.append(hah.item())

RuntimeError: shape '[-1, 6]' is invalid for input of size 256

In [370]:
# plt.plot(lri , lossi)

In [320]:
emb = C[X]  # (32, 3, 2)
h = torch.tanh(emb.view(-1, 40) @ W1 + b1)  # (32, 100)
logits = h @ W2 + b2  # (32, 27)
hah = F.cross_entropy(logits, Y)
hah

tensor(6.1381, device='mps:0', grad_fn=<NllLossBackward0>)

In [321]:
g = torch.Generator(device='mps').manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

aaaaaaaaaaaaaazaaaan.
anaaqaaaaa.
ananaaaas.
a.
aaa.
aaa.
aanaaaa.
aaaaa.
naaaaaaaannaaanaaaaaaa.
aaanaaaaaaaaaaaafa.
aaaaa.
aaaaaaaaaaaaaaaaaaaaaaaaaaaa.
a.
aaaaaaaaaaaaanaaaaaaaaanaaaaaaaaaaaaanaaaaaaaaaaaaaaaaaaa.
aaaaaaaaaaaaaa.
aaaaaaaaaaaaaaaaaaaaaa.
aa.
aaa.
aaaa.
aaaaaaaaa.
